In [18]:
import numpy as np
import pandas as pd


from sklearn.linear_model import LogisticRegression # логистическая регрессия 
from sklearn.neighbors import KNeighborsClassifier # метод K ближайших соседей
from sklearn.ensemble import RandomForestClassifier # случайный лес
from sklearn.svm import SVC # метод опорных векторов

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import classification_report

import warnings
warnings.simplefilter('ignore')

Загрузим данные для бинарной классификации по оттоку клиентов.

In [19]:
df = pd.read_csv('../data/telecom_churn.csv')
df.head()

,State,Account length,Area code,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls,Churn
0,KS,128,415,No,Yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,No,Yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,No,No,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,Yes,No,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,Yes,No,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3333 entries, 0 to 3332
Data columns (total 20 columns):
State                     3333 non-null object
Account length            3333 non-null int64
Area code                 3333 non-null int64
International plan        3333 non-null object
Voice mail plan           3333 non-null object
Number vmail messages     3333 non-null int64
Total day minutes         3333 non-null float64
Total day calls           3333 non-null int64
Total day charge          3333 non-null float64
Total eve minutes         3333 non-null float64
Total eve calls           3333 non-null int64
Total eve charge          3333 non-null float64
Total night minutes       3333 non-null float64
Total night calls         3333 non-null int64
Total night charge        3333 non-null float64
Total intl minutes        3333 non-null float64
Total intl calls          3333 non-null int64
Total intl charge         3333 non-null float64
Customer service calls    3333 non-null int64


Пропусков нет, перейдем к кодированию категориальных переменных.

In [21]:
X = df.drop('Churn', axis=1)
y = df['Churn']

In [22]:
area_code = pd.get_dummies(X['Area code'])
others_categorical = pd.get_dummies(X[['State', 'International plan', 'Voice mail plan']])
X = X.drop(
    ['Area code', 'State',
     'International plan',
     'Voice mail plan'],
    axis=1
)

X = pd.concat([X, area_code, others_categorical], axis=1)

Разделим на train и test.

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y)

Посмотрим исходное распределение классов во всей выборке:

In [24]:
df['Churn'].value_counts(normalize=True)

False    0.855086
True     0.144914
Name: Churn, dtype: float64

Если наши модели будут выдавать результат хуже 85.5 процентов правильных ответов, значит мы что-то делаем не так.

## Метод опорных векторов

In [25]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

svc = SVC(kernel='linear', C=1.0)
svc.fit(X_train_scaled, y_train)

y_train_pred = svc.predict(X_train_scaled)
print(classification_report(y_train, y_train_pred))

              precision    recall  f1-score   support

       False       0.86      1.00      0.92      1995
        True       0.00      0.00      0.00       338

    accuracy                           0.86      2333
   macro avg       0.43      0.50      0.46      2333
weighted avg       0.73      0.86      0.79      2333



In [27]:
X_test_scaled = scaler.transform(X_test)
y_test_pred = svc.predict(X_test_scaled)
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

       False       0.85      1.00      0.92       855
        True       0.00      0.00      0.00       145

    accuracy                           0.85      1000
   macro avg       0.43      0.50      0.46      1000
weighted avg       0.73      0.85      0.79      1000



__Плюсы:__
- Устойчив к переобучению, особенно в пространствах большой размерности
- Есть много ядер на выбор, некоторые из них могут работать очень хорошо на ваших данных

__Минусы:__
- Требует масштабирования признаков
- Много гиперпараметров и их влияние на конечную модель не всегда интуитивно
- Плохо масштабируется на большие датасеты (медленно обучается)

## Логистическая регрессия

In [29]:
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)

y_train_pred = log_reg.predict(X_train)
print(classification_report(y_train, y_train_pred))

              precision    recall  f1-score   support

       False       0.88      0.97      0.93      1995
        True       0.60      0.22      0.33       338

    accuracy                           0.87      2333
   macro avg       0.74      0.60      0.63      2333
weighted avg       0.84      0.87      0.84      2333



Посмотрим метрики на тестовой выборке.

In [30]:
y_test_pred = log_reg.predict(X_test)
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

       False       0.88      0.97      0.92       855
        True       0.55      0.23      0.33       145

    accuracy                           0.86      1000
   macro avg       0.72      0.60      0.63      1000
weighted avg       0.83      0.86      0.84      1000



__Плюсы:__
- Из коробки умеет выдавать вероятности, которые можно использовать
- Мало гиперпараметров для настройки
- Если переобучился, можно регуляризовать
- Использование l2-регуляризации даст важность признаков

__Минусы:__
- Может переобучаться, если признаков очень много
- Прост как пробка, потому что линейный
- Данные перед ним могут требовать масштабирования
- Может плохо работать, если есть плохие признаки

## Метод K ближайших соседей

In [41]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

knn = KNeighborsClassifier(n_neighbors=3, metric='euclidean')
knn.fit(X_train_scaled, y_train)

y_train_prediction = knn.predict(X_train_scaled)
print(classification_report(y_train, y_train_prediction))

              precision    recall  f1-score   support

       False       0.91      0.99      0.95      1995
        True       0.87      0.41      0.56       338

    accuracy                           0.91      2333
   macro avg       0.89      0.70      0.75      2333
weighted avg       0.90      0.91      0.89      2333



In [42]:
y_test_prediction = knn.predict(X_test_scaled)
print(classification_report(y_test, y_test_prediction))

              precision    recall  f1-score   support

       False       0.87      0.98      0.92       855
        True       0.53      0.14      0.22       145

    accuracy                           0.86      1000
   macro avg       0.70      0.56      0.57      1000
weighted avg       0.82      0.86      0.82      1000



__Плюсы:__
- Ленивый, не требует времени на обучение, так как просто запоминает все точки из обучающей выборки
- Предсказания до некоторой степени интерпретируемы
- Легко справляется с многоклассовой классификацией

__Минусы:__
- Данные для него надо масштабировать
- Выбор расстояния для конкретной задачи может быть не тривиальным
- Предсказание может быть дорогим удовольствием, если у вас много данных
- Плохо работает на векторах большой размерности

## Случайный лес

In [43]:
random_forest = RandomForestClassifier(n_estimators=200, max_depth=15)
random_forest.fit(X_train, y_train)

y_train_prediction = random_forest.predict(X_train)
print(classification_report(y_train, y_train_prediction))

              precision    recall  f1-score   support

       False       0.99      1.00      0.99      1995
        True       1.00      0.91      0.96       338

    accuracy                           0.99      2333
   macro avg       0.99      0.96      0.97      2333
weighted avg       0.99      0.99      0.99      2333



In [44]:
y_test_prediction = random_forest.predict(X_test)
print(classification_report(y_test, y_test_prediction))

              precision    recall  f1-score   support

       False       0.94      0.99      0.97       855
        True       0.95      0.65      0.77       145

    accuracy                           0.94      1000
   macro avg       0.95      0.82      0.87      1000
weighted avg       0.94      0.94      0.94      1000



__Плюсы:__
- Как правило дают очень хорошие результаты
- Дают важность признаков, по которой можно сделать отбор признаков
- Могут справляться с несбалансированными выборками
- Масштабируются на большие датасеты
- Очень устойчивы к переобучению
- Не нужно масштабировать данные

__Минусы:__
- Результаты может быть сложно интерпретировать
- Важность признаков не очень устойчива к шумам в исходных данных
- Плохо работают на векторах высокой размерности